In [7]:
import os
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter

In [8]:
directorio_base = os.path.abspath(os.path.join(os.getcwd(), "../.."))  # Subir dos directorios
directorio_pkl = os.path.join(directorio_base, "src", "00.data", "clustering")

# Obtener todos los archivos .pkl en el directorio de preprocesado
archivos_pkl = [f for f in os.listdir(directorio_pkl) if f.startswith("df_clustering_") and f.endswith(".pkl")]

# Si no se encuentran archivos .pkl, mostramos un mensaje
if not archivos_pkl:
    print("❌ No se encontraron archivos .pkl en el directorio.")
else:
    df_lista = []

    # Cargar todos los archivos .pkl
    for archivo in archivos_pkl:
        ruta_archivo = os.path.join(directorio_pkl, archivo)
        try:
            with open(ruta_archivo, "rb") as f:
                df_lista.append(pickle.load(f))
                print(f"✅ Archivo cargado correctamente: {archivo}")
        except FileNotFoundError:
            print(f"❌ No se encontró el archivo: {archivo}")
        except Exception as e:
            print(f"❌ Error al cargar {archivo}: {e}")

    # Concatenar todos los DataFrames en uno solo
    if df_lista:
        df = pd.concat(df_lista, ignore_index=True)
        print(f"✅ DataFrame final con {df.shape[0]} filas y {df.shape[1]} columnas")
    else:
        print("❌ No se cargaron DataFrames.")

✅ Archivo cargado correctamente: df_clustering_1.pkl
✅ Archivo cargado correctamente: df_clustering_2.pkl
✅ Archivo cargado correctamente: df_clustering_3.pkl
✅ DataFrame final con 287563 filas y 10 columnas


In [9]:
df.head()

,news_id,meneos,clicks,karma,positive_votes,negative_votes,anonymous_votes,comments,category,cluster_3
0,4032594,115,651,410,57,1,58,30,Política y Sociedad,1
1,4032568,77,982,362,50,2,27,24,Entretenimiento y Cultura,1
2,4032569,97,761,369,50,0,47,70,Política y Sociedad,1
3,4032482,141,505,352,77,1,64,53,Política y Sociedad,1
4,4032509,213,335,349,99,1,114,28,Política y Sociedad,1


In [10]:
df_clustering = df.drop(columns=['clicks', 'news_id'])

In [11]:
# Seleccionar solo las variables numéricas
num_features = ['meneos', 'karma', 'positive_votes', 'anonymous_votes', 'negative_votes', 'comments']

# Convertir a float32 antes de escalar
df_clustering[num_features] = df_clustering[num_features].astype('float32')

# Aplicar MinMaxScaler a las variables numéricas
scaler = MinMaxScaler()
df_clustering[num_features] = scaler.fit_transform(df_clustering[num_features])

# Guardar el escalador
scaler_path = "../00.data/clustering/scaler.pkl"
with open(scaler_path, "wb") as f:
    pickle.dump(scaler, f, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
# Aplicar OneHotEncoder a la columna 'category'
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_cats = encoder.fit_transform(df_clustering[['category']])

# Convertir a DataFrame
encoded_df = pd.DataFrame(encoded_cats, columns=encoder.get_feature_names_out(['category']))

# Resetear índice para asegurar alineación antes de la concatenación
df_clustering = df_clustering.reset_index(drop=True)
encoded_df = encoded_df.reset_index(drop=True)

# Unir con el dataset original
df_clustering = pd.concat([df_clustering, encoded_df], axis=1)

# Eliminar la columna categórica original
df_clustering.drop(columns=['category'], inplace=True)

# Guardar el encoder
encoder_path = "../00.data/clustering/encoder.pkl"
with open(encoder_path, "wb") as f:
    pickle.dump(encoder, f, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
# Definir X (features) e y (target)
X = df_clustering.drop(columns=['cluster_3'])
y = df_clustering['cluster_3']

# Dividir en entrenamiento y prueba (80% - 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [14]:
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "4"

In [15]:
model = NearestCentroid()
model.fit(X_train, y_train)

# Evaluar el modelo (opcional)
accuracy = model.score(X_test, y_test)
print(f"Accuracy del modelo: {accuracy:.4f}")

Accuracy del modelo: 0.2945


In [16]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.19      0.57      0.28      8868
           1       0.93      0.23      0.37     46861
           2       0.06      0.62      0.11      1784

    accuracy                           0.29     57513
   macro avg       0.39      0.47      0.25     57513
weighted avg       0.79      0.29      0.35     57513

